<a href="https://colab.research.google.com/github/Meaiie/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน

## Data House

In [74]:
import pandas as pd
import numpy as np

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
Data_House_p = '/content/drive/MyDrive/BSC_DPDM24/Midterm/Data House.xlsx'

In [77]:
Data_House = pd.read_excel(Data_House_p)
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,no,NaN,NaN,24.0,NaN,NaN,NaN,40180.0,NaN,NaN,บ้านกุดหล่ม-บ้านวังเวิน,16.139321,102.631587,0,1,45.0,NaN,NaN,390000.0,NaN,2.0,2.0,2,2.0,244.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,NaN,NaN,จากทางหลวงชนบท ขก.5069 กม.4+050,16.676156,102.917998,0,0,88.0,NaN,NaN,820000.0,NaN,1.0,1.0,1,3.0,164.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,NaN,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,11.0,11,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,เลียบบึงหนองโครต,16.459052,102.806404,0,0,61.6,NaN,NaN,5000000.0,NaN,2.0,3.0,3,2.0,203.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,16.450000,102.800680,0,0,72.3,NaN,NaN,3500000.0,NaN,2.0,3.0,2,2.0,164.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
Data_House.shape

(1656, 47)

In [79]:
# Select specific columns
selected_columns = ['renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story',
                    'number_of_bedroom','number_of_bathroom','direction','furniture','usable_area',
                    'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']
Data_House_selected = Data_House[selected_columns]

Data_House_selected.head()

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.0,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.0,3.0,820000.0,น้ำพอง,ทรายมูล,NaN
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.0,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.0,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.0,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [80]:
missing_values = Data_House_selected.isnull().sum()
missing_values

,0
renovate,211
number_of_rai,0
number_of_ngan,0
number_of_va,0
number_of_story,549
number_of_bedroom,27
number_of_bathroom,0
direction,0
furniture,0
usable_area,920


In [81]:
# prompt: เติมคำว่า 'ไม่ระบุ' แทนที่ค่าว่างในคอลัมน์ renovate,sub_district,zone
# Fill missing values in 'renovate', 'sub_district', and 'zone' columns with 'ไม่ระบุ'
Data_House['renovate'].fillna('ไม่ระบุ', inplace=True)
Data_House['sub_district'].fillna('ไม่ระบุ', inplace=True)
Data_House['zone'].fillna('ไม่ระบุ', inplace=True)

<ipython-input-81-d973d637b877>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Data_House['renovate'].fillna('ไม่ระบุ', inplace=True)
<ipython-input-81-d973d637b877>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [82]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,no,NaN,NaN,24.0,NaN,NaN,NaN,40180.0,ไม่ระบุ,NaN,บ้านกุดหล่ม-บ้านวังเวิน,16.139321,102.631587,0,1,45.0,NaN,NaN,390000.0,NaN,2.0,2.0,2,2.0,244.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,ไม่ระบุ,NaN,จากทางหลวงชนบท ขก.5069 กม.4+050,16.676156,102.917998,0,0,88.0,NaN,NaN,820000.0,NaN,1.0,1.0,1,3.0,164.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,ไม่ระบุ,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,11.0,11,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,เลียบบึงหนองโครต,16.459052,102.806404,0,0,61.6,NaN,NaN,5000000.0,NaN,2.0,3.0,3,2.0,203.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,16.450000,102.800680,0,0,72.3,NaN,NaN,3500000.0,NaN,2.0,3.0,2,2.0,164.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return outliers, lower_bound, upper_bound

numerical_cols = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
                  'number_of_bedroom', 'number_of_bathroom', 'usable_area',
                  'number_of_parking', 'sell_price']

outliers_summary = {}

for col in numerical_cols:
    outliers, lower, upper = detect_outliers_iqr(Data_House_selected[col])
    print(f"Outliers in '{col}': {len(outliers)} values")
    print(f"Lower bound: {lower}, Upper bound: {upper}")
    outliers_summary[col] = outliers
    print("-" * 20)

# หากต้องการดู outliers ทั้งหมดใน DataFrame:
outliers_df = {col: outliers_summary[col] for col in numerical_cols}


Outliers in 'number_of_rai': 111 values
Lower bound: 0.0, Upper bound: 0.0
--------------------
Outliers in 'number_of_ngan': 355 values
Lower bound: 0.0, Upper bound: 0.0
--------------------
Outliers in 'number_of_va': 65 values
Lower bound: -46.86250000000001, Upper bound: 147.4375
--------------------
Outliers in 'number_of_story': 4 values
Lower bound: -0.5, Upper bound: 3.5
--------------------
Outliers in 'number_of_bedroom': 8 values
Lower bound: -4.5, Upper bound: 7.5
--------------------
Outliers in 'number_of_bathroom': 88 values
Lower bound: -0.5, Upper bound: 3.5
--------------------
Outliers in 'usable_area': 49 values
Lower bound: -60.625, Upper bound: 420.375
--------------------
Outliers in 'number_of_parking': 16 values
Lower bound: -3.0, Upper bound: 5.0
--------------------
Outliers in 'sell_price': 106 values
Lower bound: -2400750.0, Upper bound: 7961250.0
--------------------


In [84]:
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return outliers

numerical_cols = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
                  'number_of_bedroom', 'number_of_bathroom', 'usable_area',
                  'number_of_parking', 'sell_price']

for col in numerical_cols:
    outliers = detect_outliers_iqr(Data_House_selected[col])
    print(f"Outliers in '{col}':")
    print(outliers)
    print("-" * 20)

Outliers in 'number_of_rai':
2        1
20       4
25       1
34       3
35       7
        ..
1547    25
1566     1
1574     7
1623     1
1643     1
Name: number_of_rai, Length: 111, dtype: int64
--------------------
Outliers in 'number_of_ngan':
0       1
6       1
12      3
14      1
17      3
       ..
1649    2
1650    1
1653    1
1654    1
1655    1
Name: number_of_ngan, Length: 355, dtype: int64
--------------------
Outliers in 'number_of_va':
223     250.0
267     150.0
283     400.0
287     182.0
288     260.0
        ...  
1554    343.0
1564    214.5
1565    184.7
1588    190.0
1643    780.0
Name: number_of_va, Length: 65, dtype: float64
--------------------
Outliers in 'number_of_story':
1211    32.0
1588     5.0
1597     4.0
1604     4.0
Name: number_of_story, dtype: float64
--------------------
Outliers in 'number_of_bedroom':
2       11.0
1333    10.0
1442    10.0
1517    40.0
1531    17.0
1550    36.0
1554    24.0
1588    40.0
Name: number_of_bedroom, dtype: float64
----

เพราะข้อมูลในแต่ละคอลัมน์ ค่า outlier ไม่ได้แสดงถึงความผิดพลาดของข้อมูลเสมอไป เลยดูเป็นกรณีไปค่ะ

In [85]:
# prompt: ให้ colab แสดงทุกคอลัมน์

pd.set_option("display.max_columns", None)

ในกรณีนี้ประเภทมีแค่บ้านเดี่ยวกับบ้านแฝด มี outlier ที่มีจำนวนชั้น 32 ชั้น --> คิดว่าเป็นความผิดพลาดจากการกรอกข้อมูล จึงเลือกที่จะใส่เป็นค่าว่างไปก่อนค่ะ

In [86]:
# Filter data where number_of_story is greater than 5
filtered_data = Data_House[Data_House['number_of_story'] > 5]

# Display the filtered data
filtered_data

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
1211,1088P,พฤกษาบ้านกอก บึงหนองโคตร 65 ตารางวา ตำบลบ้านเป...,พฤกษาบ้านกอก บึงหนองโคตร 65 ตารางวา ตำบลบ้านเป...,บ้านพฤกษา แอร์พอร์ต มะลิวัลย์,บริษัท พฤกษา เรียลเอสเตท จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ม.ภาค-บ้านกอก-บึงหนองโคตร,NaN,NaN,16.426829,102.810707,0,0,65.0,NaN,NaN,3700000.0,NaN,32.0,3.0,3,0.0,260.0,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
Data_House['property_type'].value_counts()

,count
property_type,
บ้านเดี่ยว,1638
บ้านแฝด,18


In [88]:
# prompt: แทนที่ข้อมูลที่มากกว่า 5 ใน คอลัมน์ number_of_story ด้วยค่าว่าง

# Replace values greater than 5 in 'number_of_story' with NaN
Data_House.loc[Data_House['number_of_story'] > 5, 'number_of_story'] = np.nan

In [89]:
# Filter data where number_of_story is greater than 5
filtered_data = Data_House[Data_House['number_of_story'] > 5]

# Display the filtered data
filtered_data

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden


ในส่วนของพื้นที่เป็นไร่และงานที่มีค่า outlier จำนวนมาก ซึ่งค่าปกติคือ 0 --> เกิดจากการที่มีพื้นที่ไม่ถึง 1 ไร่ และ ไม่ถึง 1 งาน ทำให้ค่าเป็น 0 ส่งผลให้พื้นที่อื่น ๆ ที่มีพื้นที่ถึง 1 ไร่ และ 1 งาน ขึ้นไปกลายเป็น outlier จึงตัดสินใจที่จะไม่ลบค่า outlier ออก

In [90]:
# คำนวณไร่และงานจาก number_of_va
calculated_rai = Data_House['number_of_va'] // 400
calculated_ngan = (Data_House['number_of_va'] % 400) // 100

# เติมค่าเฉพาะในแถวที่ number_of_rai และ number_of_ngan มีค่าเท่ากับ 0
Data_House.loc[Data_House['number_of_rai'] == 0, 'number_of_rai'] = calculated_rai
Data_House.loc[Data_House['number_of_ngan'] == 0, 'number_of_ngan'] = calculated_ngan

# ดูตัวอย่างข้อมูล
Data_House[['number_of_va', 'number_of_rai', 'number_of_ngan']].head()


,number_of_va,number_of_rai,number_of_ngan
0,45.0,0,1
1,88.0,0,0
2,8.0,1,0
3,61.6,0,0
4,72.3,0,0


ในส่วนของจำนวนห้องนอน เนื่องจากข้อมูลมีแค่บ้านแฝดและบ้านเดี่ยว จึงคิดว่าไม่ควรที่จะมีจำนวนห้องนอนและห้องน้ำมากไปถึง 10 ห้องหรือมากกว่านั้น จึงเลือกที่จะเติมค่าว่างในคอลัมน์จำนวนห้องนอนและห้องน้ำเป็น outlier ที่มีค่าตั้งแต่ 10 ขึ้นไป

In [91]:
# Filter the DataFrame to include only rows where 'number_of_bedroom' is greater than 10
filtered_df = Data_House[Data_House['number_of_bedroom'] > 10]

# Display the filtered data
filtered_df

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,ไม่ระบุ,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,11.0,11,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1517,30109P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,แดงใหญ่,NaN,NaN,16.501663,102.715758,0,2,239.0,NaN,NaN,35000000.0,NaN,NaN,40.0,1,NaN,NaN,NaN,ไม่ระบุ,ตกแต่งครบ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1531,30133P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,NaN,NaN,16.484438,102.799249,0,0,0.0,NaN,NaN,6900000.0,NaN,NaN,17.0,14,NaN,400.0,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,30168P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ไม่ระบุ,NaN,NaN,16.494038,102.938441,0,1,148.0,NaN,NaN,30000000.0,NaN,NaN,36.0,1,NaN,NaN,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1554,30172P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ไม่ระบุ,NaN,NaN,16.348771,102.710352,0,3,343.0,NaN,NaN,21000000.0,NaN,3.0,24.0,1,18.0,NaN,NaN,ไม่ระบุ,ตกแต่งครบ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1588,30237P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,กลางเมือง-เมืองเก่า-กุดกว้าง,NaN,NaN,16.390532,102.824919,0,1,190.0,NaN,NaN,45000000.0,NaN,5.0,40.0,1,22.0,NaN,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,มี,NaN


In [92]:
# prompt: เติมค่าว่างแทนที่ค่าที่มากว่า 9 ในคอลัมน์ number_of_bedroom และ number_of_bathroom

# Replace values greater than 9 in 'number_of_bedroom' with NaN
Data_House.loc[Data_House['number_of_bedroom'] > 9, 'number_of_bedroom'] = np.nan

# Replace values greater than 9 in 'number_of_bathroom' with NaN
Data_House.loc[Data_House['number_of_bathroom'] > 9, 'number_of_bathroom'] = np.nan

ทำเหมือนกันกับจำนวนสวน โดยที่เติมค่าว่างให้ค่าที่เป็น outlier ไปก่อน

In [93]:
# prompt: เติมค่าว่างแทนที่ค่าที่มากว่า 9 ในคอลัมน์ number_of_parking

# Replace values greater than 9 in 'number_of_parking' with NaN
Data_House.loc[Data_House['number_of_parking'] > 9, 'number_of_parking'] = np.nan

ในส่วนของพื้นที่ใช้สอย โดยปกติแล้วจะไม่มากเกินกว่า 1000 แต่ก็อาจจะมีบ้านที่มีพื้นที่ใช้สอยมากกว่า 1000 อยู่ ดูจากตัวข้อมูลแล้ว ข้อมูลไม่สอดคล้องกับพื้นที่ และจำนวนชั้นของบ้าน เลยเลือกที่จะแทนที่ด้วยค่าว่างไปก่อน

In [94]:
# prompt: เรียกดูข้อมูล ในคอลัมน์ usable_area ที่มีค่ามากกว่า 1000

# Assuming 'Data_House' DataFrame from the previous code is available.
# Filter the DataFrame for usable_area greater than 1000
filtered_df = Data_House[Data_House['usable_area'] > 1000]

# Display the filtered data
filtered_df

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
663,6946,ขายบ้าน โครงการพิมานชล2,ขายบ้าน โครงการพิมานชล2,พิมานชล2,บริษัท อีสานพิมานกรุ๊ป จำกัด,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,30/2,ริมบึงแก่นนคร,16.400254,102.838149,1,1,79.0,NaN,NaN,5500000.0,NaN,2.0,3.0,3.0,2.0,12345.0,300.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683,7712,ขายบ้านเดี่ยว 2 ชั้น (ครึ่งตึกครึ่งไม้) หนองสอ...,ขายบ้านเดี่ยว 2 ชั้น (ครึ่งตึกครึ่งไม้) หนองสอ...,NaN,NaN,ขอนแก่น,หนองสองห้อง,ดงเค็ง,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40190.0,ไม่ระบุ,NaN,สาย กม.ศูนย์-อ.หนองสองห้อง(ทล.207),15.708794,102.809742,3,0,0.0,NaN,NaN,4081000.0,NaN,0.0,0.0,1.0,0.0,1011.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,17621,ขายบ้านขอนแก่น 2 ชั้น บ้านสุขใจ ต.ทรายมูล อ.น...,ขายบ้านขอนแก่น 2 ชั้น บ้านสุขใจ ต.ทรายมูล อ.น...,NaN,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,ไม่ระบุ,ทช.สายแยกทล.2039,สายน้ำพอง-กระนวน,16.693169,102.913565,0,0,0.0,NaN,NaN,1275000.0,NaN,2.0,0.0,1.0,0.0,1872.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,17630,ขายบ้านขอนแก่น 1 ชั้น บ้านหว้าทอง ต.หว้าทอง อ....,ขายบ้านขอนแก่น 1 ชั้น บ้านหว้าทอง ต.หว้าทอง อ....,NaN,NaN,ขอนแก่น,ภูเวียง,หว้าทอง,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40150.0,ไม่ระบุ,NaN,ดอนหัน-โคกกลาง,16.724213,102.449646,0,0,0.0,NaN,NaN,1320000.0,NaN,1.0,0.0,1.0,0.0,1600.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1138,19502,ขายบ้านขอนแก่น บ้านเดี่ยว พร้อมสิ่งปลูกสร้าง 5...,ขายบ้านขอนแก่น บ้านเดี่ยว พร้อมสิ่งปลูกสร้าง 5...,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,พระลับ,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,บึงเนียม,NaN,NaN,16.409596,102.945292,10,1,17.3,NaN,NaN,10527000.0,NaN,1.0,0.0,1.0,0.0,5025.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1290,10415P,S- ขายบ้านเดี่ยว ขอนแก่น 281 ตรว. 4 ห้องนอน 4 ...,S- ขายบ้านเดี่ยว ขอนแก่น 281 ตรว. 4 ห้องนอน 4 ...,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ศิลา,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,1,0,81.0,NaN,NaN,6900000.0,NaN,2.0,4.0,4.0,0.0,1924.0,NaN,ไม่ระบุ,มีบางส่วน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1547,30157P,ขายบ้าน บ้านแฮด ขอนแก่น,ขายบ้าน บ้านแฮด ขอนแก่น,NaN,NaN,ขอนแก่น,บ้านแฮด,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ไม่ระบุ,NaN,NaN,16.156705,102.764639,25,0,0.0,NaN,NaN,5960000.0,NaN,NaN,NaN,1.0,NaN,40572.0,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1562,30192P,ขายบ้าน เมืองขอนแก่น ขอนแก่น,ขายบ้าน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ไม่ระบุ,บ้านเดี่ยว,NaN,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ในเมือง-รื่นรมย์-เทพารักษ์,NaN,NaN,16.435438,102.833502,0,0,0.0,NaN,NaN,35000000.0,NaN,2.0,3.0,2.0,NaN,1190.0,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1623,30305P,ขายที่ดิน เมืองขอนแก่น ขอนแก่น,ขายที่ดิน เมืองขอนแก่น ขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ท่าพระ,บ้านเดี่ยว,NaN,ไม่ระบุ,yes,NaN,NaN,NaN,NaN,NaN,NaN,แดงใหญ่,NaN,NaN,16.473081,102.726676,1,1,1.0,NaN,NaN,1312500.0,NaN,NaN,NaN,1.0,NaN,2800.0,NaN,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# prompt: แทนที่ค่าที่มากกว่า 1000 ในคอลัมน์ usable_area ด้วยค่าว่าง

# Replace values greater than 1000 in 'usable_area' with NaN
Data_House.loc[Data_House['usable_area'] > 1000, 'usable_area'] = np.nan

จากนั้นจะใช้วิธีการแทนที่ด้วยค่าเฉลี่ยกับข้อมูลในคอลัมน์ตัวเลขที่เราสนใจ

In [96]:
# prompt: แทนที่ค่าว่างในคอลัมน์ 'usable_area, ด้วยค่าเฉลี่ยของกลุ่มโดยแบ่งจาก property_type

# Group data by 'property_type' and calculate the mean of 'usable_area' for each group
mean_usable_area_by_type = Data_House.groupby('property_type')['usable_area'].mean()

# Fill NaN values in 'usable_area' with the mean for the corresponding 'property_type'
Data_House['usable_area'] = Data_House.apply(
    lambda row: mean_usable_area_by_type[row['property_type']] if pd.isnull(row['usable_area']) else row['usable_area'],
    axis=1
)

In [97]:
# prompt: แทนที่ค่าว่างในคอลัมน์ number_of_parking, number_of_bedroom,number_of_bathroom,sell_price ด้วยค่าเฉลี่ยของกลุ่มโดยแบ่งจาก property_type

# Group data by 'property_type' and calculate the mean for specified columns
grouped = Data_House.groupby('property_type')
mean_values = grouped[['number_of_parking', 'number_of_bedroom', 'number_of_bathroom', 'sell_price']].mean()

# Function to fill NaN values with the mean of the corresponding 'property_type' group
def fill_nan_with_group_mean(row, col):
    if pd.isnull(row[col]):
        return mean_values.loc[row['property_type'], col]
    return row[col]

# Apply the function to fill NaN values for each column
for col in ['number_of_parking', 'number_of_bedroom', 'number_of_bathroom', 'sell_price']:
    Data_House[col] = Data_House.apply(lambda row: fill_nan_with_group_mean(row, col), axis=1)

In [98]:
# prompt: แทนที่ค่าว่างในคอลัมน์ number_of_story ด้วยการทำโมเดลทำนายค่าจาก usable_area และ number_of_va

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assuming Data_House DataFrame is already loaded and preprocessed as in your code

# Prepare data for prediction
data_for_prediction = Data_House[['usable_area', 'number_of_va', 'number_of_story']].copy()
# Drop rows with NaN values in any of the columns used for prediction or target
data_for_prediction = data_for_prediction.dropna() # This line is changed to drop all rows with NaNs

X = data_for_prediction[['usable_area', 'number_of_va']]
y = data_for_prediction['number_of_story']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict missing values
missing_story_indices = Data_House[Data_House['number_of_story'].isnull()].index
X_missing = Data_House.loc[missing_story_indices, ['usable_area', 'number_of_va']]
predicted_story = model.predict(X_missing)

# Fill the missing values in the original DataFrame
Data_House.loc[missing_story_indices, 'number_of_story'] = predicted_story

In [99]:
missing_values = Data_House[['renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story', 'number_of_bedroom','number_of_bathroom','direction','furniture','usable_area', 'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']].isnull().sum()
print(missing_values)

renovate              0
number_of_rai         0
number_of_ngan        0
number_of_va          0
number_of_story       0
number_of_bedroom     0
number_of_bathroom    0
direction             0
furniture             0
usable_area           0
number_of_parking     0
sell_price            0
district              0
sub_district          0
zone                  0
dtype: int64


In [100]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,no,NaN,NaN,24.0,NaN,NaN,NaN,40180.0,ไม่ระบุ,NaN,บ้านกุดหล่ม-บ้านวังเวิน,16.139321,102.631587,0,1,45.0,NaN,NaN,390000.0,NaN,2.0,2.000000,2.00000,2.0,244.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,ไม่ระบุ,NaN,จากทางหลวงชนบท ขก.5069 กม.4+050,16.676156,102.917998,0,0,88.0,NaN,NaN,820000.0,NaN,1.0,1.000000,1.00000,3.0,164.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,ไม่ระบุ,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,1.600125,1.71315,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,เลียบบึงหนองโครต,16.459052,102.806404,0,0,61.6,NaN,NaN,5000000.0,NaN,2.0,3.000000,3.00000,2.0,203.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,16.450000,102.800680,0,0,72.3,NaN,NaN,3500000.0,NaN,2.0,3.000000,2.00000,2.0,164.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7-11 , school

In [101]:
import json

อันนี้รันไม่ทันค่ะ

In [ ]:
import json
import math
import pandas as pd

# Load JSON data
seven_eleven_file = '/content/drive/MyDrive/BSC_DPDM24/Midterm/7-eleven.json'
school_file = '/content/drive/MyDrive/BSC_DPDM24/Midterm/school.json'

with open(seven_eleven_file, 'r') as f:
    seven_eleven_data = json.load(f)

with open(school_file, 'r') as f:
    school_data = json.load(f)

# Function to calculate the distance between two coordinates (lat, lng) using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c * 1000  # Return distance in meters

# Function to count 7-Eleven and school locations within 1 km radius
def count_within_radius(data, data_list):
    count = 0
    for entry in data_list:
        if 'geometry' in entry and 'location' in entry['geometry']:
            lat2 = entry['geometry']['location']['lat']
            lon2 = entry['geometry']['location']['lng']
            for _, row in data.iterrows():
                lat1 = row['latitude']
                lon1 = row['longitude']
                # Calculate the distance between house and 7-Eleven/school
                distance = haversine(lat1, lon1, lat2, lon2)
                if distance <= 1000:  # 1 km = 1000 meters
                    count += 1
    return count

# Process 7-11 data
seven_eleven_counts = {}
for house_id, data_list in seven_eleven_data.items():
    count = count_within_radius(Data_House, data_list)
    seven_eleven_counts[house_id] = count

# Process school data
school_counts = {}
for house_id, data_list in school_data.items():
    count = count_within_radius(Data_House, data_list)
    school_counts[house_id] = count

# Add counts to Data_House DataFrame
Data_House['d1000-7/11'] = Data_House['id'].astype(str).map(seven_eleven_counts).fillna(0).astype(int)
Data_House['d1000-school'] = Data_House['id'].astype(str).map(school_counts).fillna(0).astype(int)

# Display the updated DataFrame
print(Data_House.head())


เลยรันอันนี้แทน

In [103]:
import json

# Load JSON data
seven_eleven_file = '/content/drive/MyDrive/BSC_DPDM24/Midterm/7-eleven.json'
school_file = '/content/drive/MyDrive/BSC_DPDM24/Midterm/school.json'

with open(seven_eleven_file, 'r') as f:
    seven_eleven_data = json.load(f)

with open(school_file, 'r') as f:
    school_data = json.load(f)

# Process 7-11 data
seven_eleven_counts = {}
for house_id, data_list in seven_eleven_data.items(): # Iterate through house IDs and their data
    # Assuming 'counts' key exists in each item of the data list
    seven_eleven_counts[house_id] = len(data_list) if data_list else 0 # count if data exists for a house, else 0


# Process school data
school_counts = {}
for house_id, data_list in school_data.items(): # Iterate through house IDs and their data
    # Assuming 'counts' key exists in each item of the data list
    school_counts[house_id] = len(data_list) if data_list else 0 # count if data exists for a house, else 0

# Add counts to Data_House DataFrame
Data_House['d1000-7/11'] = Data_House['id'].astype(str).map(seven_eleven_counts).fillna(0).astype(int) # map id with data and if not in data, will be 0
Data_House['d1000-school'] = Data_House['id'].astype(str).map(school_counts).fillna(0).astype(int) # map id with data and if not in data, will be 0

# Display the updated DataFrame
print(Data_House.head())

   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                    NaN  ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                    NaN  ขอนแก่น        น้ำพอง      ทรายมูล   
2                                    NaN  ขอนแก่น     มัญจาคีรี      กุดเค

In [104]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden,d1000-7/11,d1000-school
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,no,NaN,NaN,24.0,NaN,NaN,NaN,40180.0,ไม่ระบุ,NaN,บ้านกุดหล่ม-บ้านวังเวิน,16.139321,102.631587,0,1,45.0,NaN,NaN,390000.0,NaN,2.0,2.000000,2.00000,2.0,244.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,ไม่ระบุ,NaN,จากทางหลวงชนบท ขก.5069 กม.4+050,16.676156,102.917998,0,0,88.0,NaN,NaN,820000.0,NaN,1.0,1.000000,1.00000,3.0,164.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,ไม่ระบุ,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,1.600125,1.71315,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,15
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,เลียบบึงหนองโครต,16.459052,102.806404,0,0,61.6,NaN,NaN,5000000.0,NaN,2.0,3.000000,3.00000,2.0,203.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,17
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,16.450000,102.800680,0,0,72.3,NaN,NaN,3500000.0,NaN,2.0,3.000000,2.00000,2.0,164.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,17


## ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

In [105]:
ans_d = pd.read_excel('/content/drive/MyDrive/BSC_DPDM24/Midterm/คำตอบนักประเมิน(ID).xlsx')
ans_d.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [106]:
Data_House2 = Data_House.copy()

In [107]:
# prompt: สร้างคอลัมน์ 'estimator' และ 'score' โดยที่ 'estimator' ให้ดึงค่ามาจากคอลัมน์  ประเมินราคาซื้อ-ขาย ส่วน 'score' ดึงค่ามาจาก ให้คะแนนทรัพย์ โดยที่นำข้อมูลมาแค่ของ ID ผู้ประเมิน มีค่า 1 แล้วจับคู่ ID กับ id ใน Data_House

# Filter data for assessor ID 1
assessor_1_data = ans_d[ans_d['ID ผู้ประเมิน'] == 1]

# Create dictionaries to map 'id' to 'ประเมินราคาซื้อ-ขาย' and 'ให้คะแนนทรัพย์'
estimator_mapping = dict(zip(assessor_1_data['ID'], assessor_1_data['ประเมินราคาซื้อ-ขาย']))
score_mapping = dict(zip(assessor_1_data['ID'], assessor_1_data['ให้คะแนนทรัพย์']))

# Map the values to the 'Data_House' DataFrame
Data_House2['estimator'] = Data_House2['id'].map(estimator_mapping)
Data_House2['score'] = Data_House2['id'].map(score_mapping)

# Display the updated DataFrame
Data_House2.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,renovate,public utility,land type,land wide(m),land long(m),topsoil,corner,zip_code,zone,alley,street,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_unit,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,direction,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden,d1000-7/11,d1000-school,estimator,score
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,no,NaN,NaN,24.0,NaN,NaN,NaN,40180.0,ไม่ระบุ,NaN,บ้านกุดหล่ม-บ้านวังเวิน,16.139321,102.631587,0,1,45.0,NaN,NaN,390000.0,NaN,2.0,2.000000,2.00000,2.0,244.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4,600000.0,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40140.0,ไม่ระบุ,NaN,จากทางหลวงชนบท ขก.5069 กม.4+050,16.676156,102.917998,0,0,88.0,NaN,NaN,820000.0,NaN,1.0,1.000000,1.00000,3.0,164.0,0.0,ไม่ระบุ,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,750000.0,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,no,NaN,NaN,26.0,62.0,NaN,NaN,40160.0,ไม่ระบุ,NaN,ถนนกลางเมือง,16.129358,102.530670,1,0,8.0,NaN,NaN,3260000.0,NaN,1.0,1.600125,1.71315,9.0,489.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,15,2800000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,เลียบบึงหนองโครต,16.459052,102.806404,0,0,61.6,NaN,NaN,5000000.0,NaN,2.0,3.000000,3.00000,2.0,203.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,17,4500000.0,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,no,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,16.450000,102.800680,0,0,72.3,NaN,NaN,3500000.0,NaN,2.0,3.000000,2.00000,2.0,164.0,0.0,ไม่ระบุ,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,17,NaN,NaN


# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้

zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [108]:
# Group by 'zone' and calculate the mean 'sell_price'
mean_sell_price_by_zone = Data_House2.groupby('zone')['sell_price'].mean()

# Sort the zones by mean 'sell_price' in descending order and get the top 3
top_3_zones = mean_sell_price_by_zone.sort_values(ascending=False).head(3)

# Convert to integer
top_3_zones = top_3_zones.astype(int)

# Print the top 3 zones and their mean sell price
print("Top 3 zones with the highest average sell price:")
print(top_3_zones)

# Summarize the zone with the highest mean sell price
highest_zone = top_3_zones.idxmax()
highest_price = top_3_zones.max()

summary = f"The zone with the highest average sell price is '{highest_zone}' with an average price of {highest_price}."

# Print the summary
print(summary)

Top 3 zones with the highest average sell price:
zone
ในเมือง-รื่นรมย์-เทพารักษ์          7201666
โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง    5716039
ทุ่งสร้าง-ศรีจันทร์-ราชมงคล         5351464
Name: sell_price, dtype: int64
The zone with the highest average sell price is 'ในเมือง-รื่นรมย์-เทพารักษ์' with an average price of 7201666.


บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

In [109]:
# Find the house with the maximum number of 7-11s within a 1km radius
max_711_house = Data_House2['d1000-7/11'].idxmax()

# Get the sell price of that house
max_711_price = Data_House2.loc[max_711_house, 'sell_price']

# Print the result
print(f"The house with the most 7-11s in a 1km radius costs: {max_711_price}")

The house with the most 7-11s in a 1km radius costs: 5900000.0


ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [110]:
# Convert 'score' column to string type
Data_House2['score'] = Data_House2['score'].astype(str)

# Extract the first digit from the 'score' column
Data_House2['score'] = Data_House2['score'].str[0]

# Convert the extracted digit back to numeric type, handling potential errors
Data_House2['score'] = pd.to_numeric(Data_House2['score'], errors='coerce')

# Group by 'sub_district' and calculate the mean 'score'
mean_score_by_subdistrict = Data_House2.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest mean score
highest_score_subdistrict = mean_score_by_subdistrict.idxmax()
highest_mean_score = mean_score_by_subdistrict.max()

# Print the result
print(f"The sub_district with the highest average score is '{highest_score_subdistrict}' with a mean score of {highest_mean_score}.")

The sub_district with the highest average score is 'ศรีบุญเรือง' with a mean score of 4.0.


In [111]:
# Group by 'sub_district' and calculate the mean 'score'
mean_score_by_subdistrict = Data_House2.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest mean score
highest_score_subdistrict = mean_score_by_subdistrict.idxmax()
highest_mean_score = mean_score_by_subdistrict.max()

# Print the result
print(f"The sub_district with the highest average score is '{highest_score_subdistrict}' with a mean score of {highest_mean_score}.")

The sub_district with the highest average score is 'ศรีบุญเรือง' with a mean score of 4.0.
